In [1]:
from joblib import load
from sklearn.ensemble import VotingClassifier
from joblib import dump

from utils import TransactionDataset, FinalModelPerformance, TuneHyperParams


In [2]:
lr_model = load("./models_exports/logistic_regression_classifier.joblib")
knn_model = load("./models_exports/knn_classifier.joblib")
rf_model = load("./models_exports/random_forest_classifier.joblib")
nb_model = load("./models_exports/naive_bayes_classifier.joblib")
svc_model = load("./models_exports/support_vector_classifier.joblib")

In [3]:
data = TransactionDataset().get_training_test_split()

In [4]:
voting_classifier_2 = VotingClassifier(
    estimators=[
        ("lr_model", lr_model),
        ("knn_model", knn_model),
        ("rf_model", rf_model),
        ("nb_model", nb_model),
        ("svc_model", svc_model)
    ],
    voting="soft"
)

In [5]:
params = {
    "svc_model__svc_model__probability": [True],
    "voting": ["hard", "soft"],
}

In [6]:
best_model = (
    TuneHyperParams()
    .random_grid_search(
        voting_classifier_2,
        parameters=params
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_model()
)

{'svc_model__svc_model__probability': True, 'voting': 'hard'}
Best parameter (CV score: 0.669):


In [7]:
final_model_performance = FinalModelPerformance(
    model=best_model,
    data=data
)

In [8]:
final_model_performance.get_cross_validation_results()

,Metric for Training Set,Score
0,fit_time mean score,1163.984296
1,score_time mean score,0.488224
2,5-fold CV F1 Score mean score,0.668702
3,5-fold CV Accuracy mean score,0.880479
4,5-fold CV Balanced Accuracy mean score,0.835998
5,5-fold CV Matthew's Correlation Coefficient me...,0.605569


In [9]:
final_model_performance.get_final_model_performance()

,Metric for Testing Set,Score
0,F1 Score,0.696366
1,Accuracy,0.893896
2,Balanced Accuracy,0.793713
3,Matthew's Correlation Coefficient,0.638024


In [10]:
dump(final_model_performance.model, "./models_exports/voting_ensamble.joblib")

['./models_exports/voting_ensamble.joblib']